# Instalación

In [1]:
!pip install -U -q langchain langchain-community langchain-openai langchainhub

# Carga de credenciales


In [2]:
import os
from google.colab import userdata

# Recuperación segura de la API Key desde el gestor de secretos de Colab
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("Credenciales configuradas correctamente.")
except Exception as e:
    print("Error: No se encontró la clave 'OPENAI_API_KEY' en los secretos de Colab.")
    print("Por favor, agrégala en el ícono de la llave (panel izquierdo).")

Credenciales configuradas correctamente.


# Acceso a datos

In [3]:
from google.colab import drive
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

drive.mount('/content/drive')

# 1. Cargar el PDF
ruta_pdf = "/content/drive/MyDrive/Proyectos RAG/PDFs Dummy/Politica_Credito_DummyBank.pdf"
loader = PyPDFLoader(ruta_pdf)
docs = loader.load()

# 2. Fragmentación (Chunking) estratégica
# Usamos un tamaño de 800 caracteres para asegurar que las tablas de la Sección 3
# entren completas en los fragmentos
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=80)
chunks = text_splitter.split_documents(docs)

print(f"Documento fragmentado en {len(chunks)} pedazos.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Documento fragmentado en 4 pedazos.


# Creación del Vector Store (RAG)

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# 1. Definimos el modelo de embeddings (usaremos uno ligero y potente de Hugging Face)
print("Iniciando la carga del modelo de embeddings...")
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 2. Creamos la base de datos vectorial
print("Creando la base de datos vectorial en ChromaDB...")
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings_model,
    persist_directory="./chroma_db_novus"
)

print("Base de datos vectorial creada y almacenada localmente en Colab.")

Iniciando la carga del modelo de embeddings...


/tmp/ipython-input-31029966.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Creando la base de datos vectorial en ChromaDB...
Base de datos vectorial creada y almacenada localmente en Colab.


# Agente con Razonamiento

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import create_agent
from langchain_core.documents import Document

# 1. Configuración LLM

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=1000
)

# 2. Retriever optimizado para Tablas

retriever = vector_db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 10,
        "fetch_k": 20,
        "lambda_mult": 0.7
    }
)

MAX_CONTEXT_CHARS = 4000

# 3. Tool RAG

@tool
def buscar_manual(query: str) -> str:
    """
    Busca información oficial en el manual de crédito de DummyBank.
    Útil para consultar tasas, requisitos, scores y excepciones.
    """

    docs = retriever.invoke(query)

    if not docs:
        return "NO_CONTEXT_FOUND"

    filtered_docs = [
        d.page_content.strip()
        for d in docs
        if len(d.page_content.strip()) > 10
    ]

    if not filtered_docs:
        return "NO_RELEVANT_CONTEXT"

    context = "\n---\n".join(filtered_docs)

    return context[:MAX_CONTEXT_CHARS]

tools = [buscar_manual]

# 4. Prompt de Sistema (Rol: Auditor de Riesgo)

system_prompt = """
Eres un analista de riesgos experto de DummyBank.

Tus instrucciones son:
1. Basa tus respuestas ÚNICAMENTE en el contexto recuperado.
2. Si te preguntan por TASAS o MONTOS, busca con cuidado en las tablas del contexto.
3. Si el contexto menciona "Excepciones", explica los requisitos detalladamente.
4. Si la herramienta dice "NO_CONTEXT_FOUND", responde: "La información no consta en el manual."
"""

# 5. Crear agente

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt
)

print("Agente creado exitosamente.")

Agente creado exitosamente.


# Caso 1. Prueba de Extracción de Tablas

In [6]:
# Caso: Cliente con Score "Bueno" (700-749)
# Resultado esperado: Tasa 18.2%, Monto $300,000, Plazo 48 meses.
response = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Soy un cliente con un score crediticio de 720. ¿Cuál es la tasa de interés, el monto máximo que me corresponde según la tabla, y a que plazo?"
        }
    ]
})

print(response["messages"][-1].content)

Con un score crediticio de 720, que se clasifica como "Bueno", los detalles son los siguientes:

- **Tasa de interés anual (TIA):** 18.2%
- **Monto máximo:** $300,000 MXN
- **Plazo máximo:** 48 meses


# Caso 2. Prueba de Lógica Negativa

In [7]:
# Caso: Cliente con Score "Limitado" (<650) pero fuera del rango de excepción
# Resultado esperado: Rechazo Automático.
response = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Mi score es de 600. ¿Puedo solicitar un crédito?"
        }
    ]
})

print(response["messages"][-1].content)

Con un score de 600, no cumples con el mínimo requerido para las categorías de crédito disponibles en DummyBank. Según la matriz de riesgo, el score mínimo para acceder a un crédito es de 650, que corresponde a la categoría "Regular".


# Caso 3. Prueba de Lógica Compleja / Excepciones

In [8]:
# Caso: Cliente de excepción (Score 630-649)
# Resultado esperado: Aprobado bajo excepción, requiere firma del Subdirector de Crédito.
response = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Tengo un score de 640, pero llevo 6 años siendo cliente del banco y nunca me he atrasado en pagos. ¿Hay alguna excepción para mí?"
        }
    ]
})

print(response["messages"][-1].content)

Sí, hay una excepción para clientes como tú. Los clientes con un score entre 630 y 649 pueden ser aprobados bajo excepción si demuestran una relación comercial previa con el banco mayor a 5 años y no presentan atrasos en los últimos 24 meses. Esta aprobación requiere la firma de un gerente.


# Caso 4. Prueba de Cumplimiento / AML

In [9]:
# Caso: Monto alto (> $100,000)
# Resultado esperado: Debe mencionar el formato "Declaración de Origen de Fondos".
response = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Voy a pedir $150,000 MXN. ¿Debo presentar alguna declaración de prevención de lavado de dinero?"
        }
    ]
})
print(response["messages"][-1].content)

Sí, dado que la transacción supera los $100,000 MXN, es necesario presentar el formato de "Declaración de Origen de Fondos" como parte de las medidas de prevención de lavado de dinero.


#Caso 5. Prueba de Requisitos

In [10]:
# Caso: Requisitos generales
# Resultado esperado: Edad 21-70, Ingreso $15k, Antigüedad 12 meses.
response = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "¿Cuáles son la edad mínima y el ingreso mensual requerido para el crédito?"
        }
    ]
})

print(response["messages"][-1].content)

La edad mínima requerida para solicitar un crédito es de 21 años, y el ingreso mensual mínimo comprobable debe ser de $15,000 MXN netos.


# Caso 6. Prueba de "Fuera de Dominio" (Negative Testing)

In [11]:
# El manual es exclusivo de Créditos Personales.
# Preguntamos por un producto coherente (Hipotecas) pero inexistente en el documento.
response = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "¿Cuál es la tasa de interés para un crédito hipotecario o de vivienda?"
        }
    ]
})

print(response["messages"][-1].content)

La información no consta en el manual.
